1. Carregar os dados dos discursos

In [44]:
import pandas as pd
import sqlite3

# Caminho ao banco de dados SQLite
db_path = "../../../DiscursosSenadores_02_05_2025_analisado.sqlite"

# Conectar ao banco
conn = sqlite3.connect(db_path)

# Carregar os dados da tabela 'AnaliseCorpusTodo'
query = """
SELECT
    a.*,
    d.TextoResumo,
    d.Indexacao,
    d.DataPronunciamento,
    d.SiglaCasaPronunciamento
FROM AnaliseCorpusTodo AS a
LEFT JOIN Discursos AS d
    ON a.CodigoPronunciamento = d.CodigoPronunciamento
WHERE d.SiglaCasaPronunciamento = 'SF'
"""
df_discursos = pd.read_sql_query(query, conn)

# Fechar conexão
conn.close()

print(df_discursos.head())


   CodigoPronunciamento                                      AvalConclusao  \
0                366328                                               None   
1                366329                                               None   
2                366330                                               None   
3                366331  Por causa disso, o orador defende a necessidad...   
4                366332                                               None   

                                      AvalImplicacao  \
0                                               None   
1                                               None   
2                                               None   
3  isso significa que a Constituição busca promov...   
4                                               None   

                                      AvalPredicacao  \
0                                               None   
1                                               None   
2                         

2. Definir coluna de análise

In [45]:
coluna = input("Digite a coluna da tabela AnaliseCorpusTodo a ser topicalizada. Caso seja mais de uma coluna, prepare uma coluna combinada.")

3. Carregar embeddings FAISS e os códigos

In [46]:
import numpy as np
import pandas as pd
import faiss

# Caminhos aos arquivos
index_path = f"../../../../data/discursos/embeddings/discursos_{coluna}.index"
codigos_path = f"../../../../data/discursos/embeddings/codigos_{coluna}.npy"

# Carregar códigos (ids dos documentos)
codigos = np.load(codigos_path)

# Carregar índice FAISS e extrair embeddings
faiss_index = faiss.read_index(index_path)
embeddings = faiss_index.reconstruct_n(0, faiss_index.ntotal)


4. Verificação e alinhamento embeddings ↔ textos originais

In [48]:
# Validação rápida inicial
print("Embeddings shape:", embeddings.shape)
print("Codigos shape:", codigos.shape)

# Criar DataFrame para alinhamento seguro
df_embeddings = pd.DataFrame({
    "CodigoPronunciamento": codigos,
    "embedding_index": np.arange(len(codigos))
})

# Garantir alinhamento com df_final_valido (já preparado anteriormente)
df_final_valido = pd.merge(
    df_final_valido, df_embeddings,
    on="CodigoPronunciamento", how="inner"
)

# Selecionar embeddings válidos
embeddings_validos = embeddings[df_final_valido["embedding_index"]]

# Validação final
print(f"Nº documentos válidos alinhados: {len(df_final_valido)}")
print(f"Shape embeddings válidos alinhados: {embeddings_validos.shape}")


Embeddings shape: (15531, 3072)
Codigos shape: (15531,)
Nº documentos válidos alinhados: 11437
Shape embeddings válidos alinhados: (11437, 3072)


In [49]:
df_final_valido.head()

,CodigoPronunciamento,AvalConclusao,AvalImplicacao,AvalPredicacao,AvalTrecho,BM25_CorpusTodo_Constituição,MencionaConstituicao,NormConclusao,NormImplicacao,NormPredicacao,...,SumarioConstituicao,TFIDF_CorpusTodo_Constituição,TopicosConstituicao,TrechosConstituicao,TextoResumo,Indexacao,DataPronunciamento,embedding_index_x,embedding_index_y,embedding_index
0,425993,"Por causa disso, o orador defende que a luta d...",Isso significa que as conquistas sociais alcan...,a Constituição de 1988 representa parcas conqu...,"Implícito: ""...tudo isso esteja acontecendo pa...",1.518390,1.0,"Por causa disso, o orador critica as propostas...","isso implica que as ações do Estado, incluindo...",o Estado democrático de direito deve ser prese...,...,A oradora defende que as conquistas sociais ob...,0.017537,Críticas à reforma trabalhista e defesa dos di...,"['...nós vamos ter retrocessos nas parcas, par...","Críticas à condução da Operação Lava Jato, bem...","CRITICA, AUSENCIA, IMPARCIALIDADE, OPERAÇÃO, P...",2016-09-27,0,0,0
1,491530,"Por causa disso, o orador defende que o Senado...",isso significa que há uma crise de confiança n...,a Constituição é desrespeitada por alguns mini...,"""Mas alguns ministros são os primeiros a desre...",2.356893,1.0,"Por causa disso, o orador critica a atuação de...",isso implica que o STF deve agir conforme os p...,a Constituição deve ser defendida e ser o guar...,...,O orador critica a atuação de alguns ministros...,0.038622,Críticas ao Supremo Tribunal Federal por desre...,"['Deveria ser assim. Mas, infelizmente, existe...",Reflexão sobre o comportamento de membros do P...,"COMENTARIO, COMPORTAMENTO, MEMBROS, JUDICIARIO...",2022-06-02,1,1,1
2,491532,"Por causa disso, o orador apoia a PEC nº 11, d...","Isso significa que, sem ajustes, algumas leis ...",a Constituição precisa ser ajustada para asseg...,nós deveríamos sanar esse problema da constitu...,2.897702,1.0,"Por causa disso, o orador defende a aprovação ...",Isso implica que qualquer legislação que afete...,a Constituição não permite que o Congresso Nac...,...,O orador expressa uma visão positiva sobre a C...,0.090180,Emendas constitucionais e piso salarial para p...,"['""...foi apresentada justamente a PEC 11. Enq...",Como Relator - Para proferir parecer sobre a *...,"RELATOR, PARECER, PROPOSTA DE EMENDA A CONSTIT...",2022-06-02,2,2,2
3,426006,"Por causa disso, o orador sugere que a defesa ...",Isso significa que a família deve ser central ...,A Constituição é vista como um documento que r...,A defesa da família deve constituir o princípi...,2.096406,1.0,"Por causa disso, o orador defende que política...",Isso implica que o Estado deve garantir proteç...,A Constituição estabelece que a família é a ba...,...,O orador exalta a Constituição de 1988 ao menc...,0.033112,Direitos das crianças e adolescentes na Consti...,"[""O art. 226 da nossa Constituição Federal dis...",Discurso de posse de S. Exa. como primeiro sup...,"DISCURSO, POSSE, ORADOR, SUPLENTE, ACIR GURGAC...",2016-09-19,3,3,3
4,491543,"Por causa disso, o orador apoia o projeto de l...",None,None,None,2.084855,1.0,"Por causa disso, o orador apoia a inclusão do ...",isso implica que o piso salarial da enfermagem...,a Constituição pode incluir o piso salarial da...,...,A oradora expressa apoio à inclusão do piso sa...,0.029980,Emendas constitucionais e piso salarial para p...,"['nós estamos colocando na Constituição, gente...","Discussão sobre a ***PEC*** (***PEC***) n° 11,...","DISCUSSÃO, PROPOSTA DE EMENDA A CONSTITUIÇÃO (...",2022-06-02,4,4,4


5. Garantir que datas estejam no formato adequado

In [50]:
df_final_valido["DataPronunciamento"] = pd.to_datetime(df_final_valido["DataPronunciamento"], errors='coerce')

# Checar problemas de datas
print(df_final_valido["DataPronunciamento"].isnull().sum())


0


In [51]:
len(df_final_valido)

11437

6. Salvar dataset preparado

In [52]:
# Exportar resultados finais de maneira clara e definitiva
df_final_valido.to_parquet(f"dataset_preparado_{coluna}.parquet", index=False)
np.save(f"embeddings_validos_{coluna}.npy", embeddings_validos)